# Miniflow
MiniFlow has the makings of becoming a powerful deep learning tool. It is entirely possible to classify something like the [MNIST](http://yann.lecun.com/exdb/mnist/) database with MiniFlow.



## Introduction

In this lab, you’ll build a library called MiniFlow which will be your own version of [TensorFlow](http://tensorflow.org/) - a deep learning library. Another deep learning library is [Keras](https://keras.io/).

The goal of this lab is to demystify two concepts at the heart of neural networks - **backpropagation and differentiable graphs**.

**Backpropagation** is the process by which neural networks update the weights of the network over time. You can see it [here](../../nd/ml/notes/supervised-learning/artificial-neural-networks/artificial-neural-networks.ipynb).

**Differentiable graphs** are graphs where the nodes are differentiable functions. They are also useful as visual aids for understanding and calculating complicated derivatives. This is the fundamental abstraction of TensorFlow - it's a framework for creating differentiable graphs.

With graphs and backpropagation, you will be able to create your own nodes and properly compute the derivatives. Even more importantly, you will be able to think and reason in terms of these graphs.








## Graphs
What is a Neural Network?
![example-neural-network](./images/example-neural-network.png)

A neural network is a graph of mathematical functions such as [linear combinations](https://en.wikipedia.org/wiki/Linear_combination) and activation functions. The graph consists of **nodes**, and **edges**.

Nodes in each layer (except for nodes in the input layer) perform mathematical functions using inputs from nodes in the previous layers. For example, a node could represent f(x,y)=x+y, where x and y are input values from nodes in the previous layer.

Similarly, each node creates an output value which may be passed to nodes in the next layer. The output value from the output layer does not get passed to a future layer (last layer!)

Layers between the input layer and the output layer are called **hidden layers**.

The edges in the graph describe the connections between the nodes, along which the values flow from one layer to the next. These edges can also apply operations to the values that flow along them, such as multiplying by weights, adding biases, etc.. MiniFlow won't use a special class for edges. Instead, its nodes will perform both their own calculations and those of their input edges. This will be more clear as you go through these lessons.

### Forward Propagation

By propagating values from the first layer (the input layer) through all the mathematical functions represented by each node, the network outputs a value. This process is called a **forward pass**.

### Graphs

The nodes and edges create a graph structure. Though the example above is fairly simple, it isn't hard to imagine that increasingly complex graphs can calculate . . . well . . . *almost anything*.

There are generally two steps to create neural networks:

1. Define the graph of nodes and edges.
2. Propagate values through the graph.

`MiniFlow` works the same way. You'll define the nodes and edges of your network with one method and then propagate values through the graph with another method. `MiniFlow` comes with some starter code to help you out.

## Miniflow Architecture
Let's consider how to implement this graph structure in `MiniFlow`. We'll use a Python class to represent a generic node.

```
class Node(object):
    def __init__(self):
        # Properties will go here!

```

We know that each node might receive input from multiple other nodes. We also know that each node creates a single output, which will likely be passed to other nodes. Let's add two lists: one to store references to the inbound nodes, and the other to store references to the outbound nodes.

```
class Node(object):
    def __init__(self, inbound_nodes=[]):
        # Node(s) from which this Node receives values
        self.inbound_nodes = inbound_nodes
        # Node(s) to which this Node passes values
        self.outbound_nodes = []
        # For each inbound Node here, add this Node as an outbound Node to _that_ Node.
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)

```

Each node will eventually calculate a value that represents its output. Let's initialize the `value` to `None` to indicate that it exists but hasn't been set yet.

```
class Node(object):
    def __init__(self, inbound_nodes=[]):
        # Node(s) from which this Node receives values
        self.inbound_nodes = inbound_nodes
        # Node(s) to which this Node passes values
        self.outbound_nodes = []
        # For each inbound Node here, add this Node as an outbound Node to _that_ Node.
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        # A calculated value
        self.value = None

```

Each node will need to be able to pass values forward and perform backpropagation (more on that later). For now, let's add a placeholder method for forward propagation. We'll deal with backpropagation later on.

```
class Node(object):
    def __init__(self, inbound_nodes=[]):
        # Node(s) from which this Node receives values
        self.inbound_nodes = inbound_nodes
        # Node(s) to which this Node passes values
        self.outbound_nodes = []
        # For each inbound Node here, add this Node as an outbound Node to _that_ Node.
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        # A calculated value
        self.value = None

    def forward(self):
        """
        Forward propagation.

        Compute the output value based on `inbound_nodes` and
        store the result in self.value.
        """
        raise NotImplemented

```

### Nodes that Calculate

While `Node` defines the base set of properties that every node holds, only specialized [subclasses](https://docs.python.org/3/tutorial/classes.html#inheritance) of `Node` will end up in the graph. As part of this lab, you'll build the subclasses of `Node` that can perform calculations and hold values. For example, consider the `Input` subclass of `Node`.

```
class Input(Node):
    def __init__(self):
        # An Input node has no inbound nodes,
        # so no need to pass anything to the Node instantiator.
        Node.__init__(self)

    # NOTE: Input node is the only node where the value
    # may be passed as an argument to forward().
    #
    # All other node implementations should get the value
    # of the previous node from self.inbound_nodes
    #
    # Example:
    # val0 = self.inbound_nodes[0].value
    def forward(self, value=None):
        # Overwrite the value if one is passed in.
        if value is not None:
            self.value = value

```

Unlike the other subclasses of `Node`, the `Input` subclass does not actually calculate anything. The `Input` subclass just holds a `value`, such as a data feature or a model parameter (weight/bias).

You can set `value` either explicitly or with the `forward()` method. This value is then fed through the rest of the neural network.

### The Add Subclass

`Add`, which is another subclass of `Node`, actually can perform a calculation (addition).

```
class Add(Node):
    def __init__(self, x, y):
        Node.__init__(self, [x, y])

    def forward(self):
        """
        You'll be writing code here in the next quiz!
        """

```

Notice the difference in the `__init__` method, `Add.__init__(self, [x, y])`. Unlike the `Input` class, which has no inbound nodes, the `Add` class takes 2 inbound nodes, `x` and `y`, and adds the values of those nodes.

To sum up:
```
        Node (input - inbound nodes, output - outbound nodes)
        /\
       /  \
      /    \
    Input   Add (Input class has no inbound nodes, but Add class has 2 inbound nodes)
```

  


## 1 - Forward Propagation
`MiniFlow` has two methods to help you define and then run values through your graphs: `topological_sort()` and `forward_pass()`.

In order to define your network, you'll need to define the order of operations for your nodes. Given that the input to some node depends on the outputs of others, you need to flatten the graph in such a way where all the input dependencies for each node are resolved before trying to run its calculation. This is a technique called a [topological sort](https://en.wikipedia.org/wiki/Topological_sorting).

The `topological_sort()` function implements topological sorting using [Kahn's Algorithm](https://en.wikipedia.org/wiki/Topological_sorting#Kahn.27s_algorithm). The details of this method are not important, the result is; `topological_sort()` returns a sorted list of nodes in which all of the calculations can run in series. `topological_sort()` takes in a `feed_dict`, which is how we initially set a value for an `Input` node. The `feed_dict` is represented by the Python dictionary data structure. Here's an example use case:

```
# Define 2 `Input` nodes.
x, y = Input(), Input()

# Define an `Add` node, the two above`Input` nodes being the input.
add = Add(x, y)

# The value of `x` and `y` will be set to 10 and 20 respectively.
feed_dict = {x: 10, y: 20}

# Sort the nodes with topological sort.
sorted_nodes = topological_sort(feed_dict=feed_dict)

```

### Setup

Review `nn.py` and `miniflow.py`.

The neural network architecture is already there for you in nn.py. It's your job to finish `MiniFlow` to make it work.

For this quiz, I want you to:

1. Open `nn.py` below. **You don't need to change anything.** I just want you to see how `MiniFlow` works.
2. Open `miniflow.py`. **Finish the forward method on the Add class. All that's required to pass this quiz is a correct implementation of forward.**
3. Test your network by hitting "Test Run!" When the output looks right, hit "Submit!"

## Convention for running the code
Note that nn.py contains the runner code and Miniflow has the lib. So, please refer the code. Here we will just run the code.

In [13]:
from forward_propagation.nn import *
from forward_propagation.miniflow import *


## 2 - Learning and Loss
Like MiniFlow in its current state, neural networks take inputs and produce outputs. But unlike MiniFlow in its current state, neural networks can improve the accuracy of their outputs over time (it's hard to imagine improving the accuracy of Add over time!). To explore why accuracy matters, I want you to first implement a trickier (and more useful!) node than Add: the Linear node.


As described by Charles and Michael, a Neuron calculates the weighted sum of its inputs.

Think back to Neural Networks lesson with Charles and Michael. A simple artificial neuron depends on three components:

- inputs, *x* (vector)
- weights, *w* (vector)
- bias, *b* (scalar)

The output, *o*, is just the weighted sum of the inputs plus the bias:


o =  Σx<sub>i</sub>w<sub>i</sub> + b


Equation (1)

Remember, by varying the weights, you can vary the amount of influence any given input has on the output. The learning aspect of neural networks takes place during a process known as backpropagation. In backpropogation, the network modifies the weights to improve the network's output accuracy. You'll be applying all of this shortly.

In this next quiz, you'll try to build a linear neuron that generates an output by applying a simplified version of Equation (1). `Linear` should take an list of inbound nodes of length *n*, a list of weights of length *n*, and a bias.

### Instructions

1. Open nn.py below. Read through the neural network to see the expected output of `Linear`.
2. Open miniflow.py below. Modify `Linear`, which is a subclass of `Node`, to generate an output with Equation (1).


## 3 - Linear Transform

## 4 - Sigmoid Function

## 5 - Cost

## 6 - Gradient Descent

## 7 - Backpropogation

## 8 - Stochastic Gradient Descent